# Keras + Tensorflow Example: Pixel Regression with GPU

## Can we recover an image by learning a deep regression map from pixels $(x,y)$ to colors $(r,g,b)$?

You bet! The idea is to use a deep learning (DL) solution to do a deep regression to learn a mapping between pixel locations and RGB colors, with the goal of generating an image one pixel at a time. This means that if the dimensions of the target image are X-by-Y, then it is necessary to run the network X*Y. If the image is 100-by-100, then 10,000 training iterations are executed.

**Keras with TensorFlow** as a backend creates a working model for this project. Pythonistas rejoice: Keras functional API is a great abstraction for several **Deep Learning** frameworks including Tensorflow to help define complex models, such as multi-output models, directed acyclic graphs, or models with shared layers.

The Sequential model is probably a better choice to implement such a network, but it helps to start with something surprisingly simple.

Using the Model class:

- A layer instance is callable (on a tensor), and it returns a tensor.
- Input tensor(s) and output tensor(s) can then be used to define a model.
- Such a model can be trained using the Keras Sequential models.

Our target image will be the Mona Lisa.

## Let's Get Started!

First, lets use **matplotlib** to import and render the Mona Lisa from a standard *jpg format.


In [ ]:
import matplotlib.image as mpimg
import matplotlib.pylab as plt
import numpy as np
%matplotlib inline

im = mpimg.imread("monalisa.jpg")

plt.imshow(im)
plt.show()
im.shape

## Training and Validation Data

Our **training** dataset will be composed of pixels locations and input and pixel values as output are used for **validation**. Here we are loop through the image shape and create floating point arrays for a two dimensional matrix of where the pixels are located:

In [ ]:
X_train = []
Y_train = []
for i in range(im.shape[0]):
    for j in range(im.shape[1]):
        X_train.append([float(i),float(j)])
        Y_train.append(im[i][j])
        
X_train = np.array(X_train)
Y_train = np.array(Y_train)
print('Samples:', X_train.shape[0])
print('(x,y):', X_train[0],'\n', '(r,g,b):',Y_train[0])

## Objective 

Our objective is to create a model that is able to reconstruct the image in R, G, B format.

There is a lot of information about Keras and Tensorflow options online. To keep this example simple, let's just assume some standard ways of defining the model:

- **Keras Sequential**: a linear stack of layers. We do this by passing a list of layer instances to the constructor.
- **Add Layers**: here we are using the standard add() method to add additional layers.
- **Define Input Shape**: we will use the Dense 2D layer and define it as a 2D shape with the *input_dim* parameter.

This example uses a "plain vanilla" neural network, known as [MLP for binary classification](https://keras.io/getting-started/sequential-model-guide/).

In [ ]:
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.optimizers import Adam, RMSprop

model = Sequential()

model.add(Dense(500, kernel_initializer="uniform", input_dim=2))
model.add(Activation('relu'))

model.add(Dense(500, kernel_initializer="uniform", input_dim=2))
model.add(Activation('relu'))

model.add(Dense(500, kernel_initializer="uniform", input_dim=2))
model.add(Activation('relu'))

model.add(Dense(500, kernel_initializer="uniform", input_dim=2))
model.add(Activation('relu'))

model.add(Dense(500, kernel_initializer="uniform", input_dim=2))
model.add(Activation('relu'))

model.add(Dense(3, kernel_initializer="uniform", input_dim=2))
model.add(Activation('linear'))

model.summary()

model.compile(loss='mean_squared_error',
              optimizer=Adam(),
              metrics=['accuracy'])

# Why use Adam Optimizer?
# Kingma et al. [1] show that its bias-correction helps Adam slightly outperform RMSprop 
# towards the end of optimization as gradients become sparser. Insofar, Adam might be the best overall choice.
# [1]Kingma, D. P., & Ba, J. L. (2015). Adam: a Method for Stochastic Optimization. 
# International Conference on Learning Representations, 1–13.

In [ ]:
# use this cell to find the best model architecture
model.fit(X_train, Y_train, epochs=1000, shuffle=True, verbose=1, batch_size=500)
Y = model.predict(X_train, batch_size=10000)
k = 0
im_out = im[:]
for i in range(im.shape[0]):
    for j in range(im.shape[1]):
        im_out[i,j]= Y[k]
        k += 1
        
print("Mona Lisa by DL")
plt.imshow(im_out)
plt.show()

In [ ]:
plt.imshow(im_out)
plt.show()